<a href="https://colab.research.google.com/github/MaratNaz12/ML-DL/blob/main/cifar10_class_Resnet9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Import section**

In [2]:
import os
import torch
import torchvision
import tarfile
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
from torch.utils.data import random_split
from torchvision.utils import make_grid

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.facecolor'] = '#ffffff'

### **Dara preapring**

Downloading data

In [ ]:
dataset_url = "https://s3.amazonaws.com/fast-ai-imageclas/cifar10.tgz"
download_url(dataset_url, '.')

with tarfile.open('./cifar10.tgz', 'r:gz') as tar:
    tar.extractall(path='./data')
data_dir = './data/cifar10'

Trsnaforming and splitting into test/train

In [7]:
#mean and standard deviation of cifar10 dataset
stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

#tranform model from images to tensor, a bit of data augmentation, normalization
train_transform= tt.Compose([tt.RandomCrop(32, padding = 4, padding_mode = 'reflect'),
                             tt.RandomHorizontalFlip(),
                             tt.ToTensor(),
                             tt.Normalize(*stats, inplace = True)])

valid_transform = tt.Compose([tt.ToTensor(),tt.Normalize(*stats,inplace = True)])


#splitting data into training/validation/testing datasets

train_dataset= ImageFolder(data_dir+'/train', train_transform) 
'''+ ImageFolder(data_dir+'/train', valid_transform)'''
test_dataset = ImageFolder(data_dir+'/test',  valid_transform) 

valid_size = 5000
train_dataset, valid_dataset = random_split(train_dataset, [len(train_dataset) - valid_size, valid_size])

batch_size = 400
train_dataset = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=2, pin_memory=True)
valid_dataset = DataLoader(valid_dataset, batch_size * 2, num_workers=2, pin_memory=True)
test_dataset = DataLoader(test_dataset,batch_size,num_workers = 2, pin_memory = True )

### **GPU activation**

In [15]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)
        
device = get_default_device()
train_dataset = DeviceDataLoader(train_dataset, device)
valid_dataset= DeviceDataLoader(valid_dataset, device)


### **NN model**

Metircs

In [16]:
#metrics
def accuracy(preds, labels):
  _, preds = torch.max(preds, dim=1)
  return torch.tensor(torch.sum(preds == labels).item() / len(preds))

Model base

In [23]:
class ImageClassificationBase(nn.Module):

  #train loss on batch
  def training_step(self,batch):
    images,labels = batch
    preds = self(images)
    loss = F.cross_entropy(preds, labels)
    return loss
  #validation loss and accuracy on batch
  def validation_step(self,batch):
    images, labels = batch
    preds = self(images)
    loss = F.cross_entropy(preds,labels)
    acc = accuracy(preds,labels)
    return {'val_loss': loss.detach(), 'val_acc': acc}

  #mean validation loss and accuracy on validation dataset
  def validation_epoch_end(self,outputs):
    epoch_loss = torch.tensor([x['val_loss'] for x in outputs]).mean()
    epoch_acc =  torch.tensor([x['val_acc'] for x in outputs]).mean()
    return {'val_loss' :epoch_loss,'val_acc' :epoch_acc}
    
  #printing res of epoch
  def epoch_end(self, epoch, result):
        print("Epoch [{}],  train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

Covolution layer constructor
( helping block)


In [24]:
def conv_block(in_chanels, out_chanels, pool = False):
  layers = [nn.Conv2d(in_chanels, out_chanels,kernel_size=3,padding = 1),
            nn.BatchNorm2d(out_chanels), 
            nn.ReLU(inplace=True)]
  if pool: layers.append(nn.MaxPool2d(2))
  return nn.Sequential(*layers)

Model ResNet9

In [25]:
class ResNet9(ImageClassificationBase):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        
        self.conv1 = conv_block(in_channels, 64)
        self.conv2 = conv_block(64, 128, pool=True)
        self.res1 = nn.Sequential(conv_block(128, 128), conv_block(128, 128))
        
        self.conv3 = conv_block(128, 256, pool=True)
        self.conv4 = conv_block(256, 512, pool=True)
        self.res2 = nn.Sequential(conv_block(512, 512), conv_block(512, 512))
        
        self.classifier = nn.Sequential(nn.MaxPool2d(4), 
                                        nn.Flatten(), 
                                        nn.Dropout(0.2),
                                        nn.Linear(512, num_classes))
        
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

        
model = to_device(ResNet9(3, 10), device)

### **Training func**

In [30]:
def evaluate(model, valid_dataset):
  model.eval()
  outputs = [model.validation_step(batch) for batch in valid_dataset]
  return model.validation_epoch_end(outputs)

def fit(epoch_num, lr, model,train_dataset, valid_dataset, grad_clip ,weight_decay, opt):
  # torch.cuda.empty_cache()
  opt = opt(model.parameters(), lr, weight_decay=weight_decay)
  history = []
  for epoch in range(epoch_num):
    model.train()
    train_losses = []

    for batch in train_dataset:

      loss = model.training_step(batch) 
      train_losses.append(loss.detach())
      loss.backward()
      nn.utils.clip_grad_value_(model.parameters(), grad_clip)
      opt.step()
      opt.zero_grad()

    result = evaluate(model, valid_dataset)
    result['train_loss'] = torch.tensor(train_losses).mean().item()
    model.epoch_end(epoch, result)
    history.append(result)

  return history


### Train and Test

In [27]:
history = []

grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

In [ ]:
lr = 0.004
epochs = 13
history += fit(epochs, lr, model, train_dataset, valid_dataset, grad_clip, weight_decay, opt_func)

In [ ]:
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs')
    
plot_accuracies(history)

In [ ]:
def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');

plot_losses(history)

In [ ]:
test_dataset= DeviceDataLoader(test_dataset, device)
outputs = [model.validation_step(batch) for batch in valid_dataset]
test_loss = torch.tensor([x['val_loss'] for x in outputs]).mean()
test_acc =  torch.tensor([x['val_acc'] for x in outputs]).mean()
print("Test: test_loss: {:.4f}, test_acc: {:.4f}".format(
            test_loss,test_acc))

